In [1]:
"""南大实习day3"""
import pandas as pd
import numpy as np
from NSCOracle import *

In [10]:
from NSCDataProcess import *

In [7]:
sql_ = """
with tmp_innercode as (
select 
innercode,secucode,LISTEDSECTOR 
from secumain
where secucategory in (1)
AND SECUMARKET IN (83,90)
and LISTEDSECTOR IN (1,6,7)
AND LISTEDSTATE IN (1)
GROUP BY innercode,secucode,LISTEDSECTOR 
)

SELECT TRADINGDAY ,t2.SECUCODE ,CLOSEPRICE ,LISTEDSECTOR 
FROM dm_frontadjustdailyquote t1,tmp_innercode t2
WHERE TRADINGDAY BETWEEN '20200101' AND '20201231'
AND t1.innercode = t2.innercode 
"""
df_hq = get_data_from_oracle(qt_cursor,sql_)

In [8]:
df_hq.head()

tradingday secucode  closeprice  listedsector
0    20200714   600058        7.11             1
1    20200714   600745      155.17             1
2    20200714   600750       13.98             1
3    20200714   600026        8.25             1
4    20200714   600771       15.01             1

In [9]:
df_hq.dtypes

tradingday        int64
secucode         object
closeprice      float64
listedsector      int64
dtype: object

In [22]:
df_hq['tradingday'] = df_hq['tradingday'].astype(str)

In [11]:
closeprice = stack2dataframe(df_hq,'tradingday','secucode','closeprice')
changepct = closeprice/closeprice.shift(1) -1

In [14]:
df_changepct =changepct.stack().reset_index().rename(columns = {0:'changepct'})

In [15]:
df_changepct.head()

tradingday secucode  changepct
0    20200103   000001   0.018376
1    20200103   000002  -0.015663
2    20200103   000004   0.000896
3    20200103   000005   0.003185
4    20200103   000006   0.014815

In [16]:
df_hq = pd.merge(df_hq,df_changepct,on = ['tradingday','secucode'],how = 'left')

In [21]:
df_hq.dtypes

tradingday        int64
secucode         object
closeprice      float64
listedsector      int64
changepct       float64
dtype: object

In [29]:
df_hq = df_hq.dropna()

In [33]:
df_hq.isnull().sum()

tradingday      0
secucode        0
closeprice      0
listedsector    0
changepct       0
date_dt         0
dtype: int64

In [46]:
df_hq['date_dt'] = df_hq.apply(
    lambda x:1 if x['changepct']<=-0.098 and x['tradingday']<'20200824'
              else 1 if  x['changepct']<=-0.198 and x['tradingday']>='20200824' and x['listedsector']==6
              else 1 if  x['changepct']<=-0.098 and x['tradingday']>='20200824' and x['listedsector']!=6
              else 0 ,axis=1)

In [45]:
df_hq[(df_hq['date_dt']==1)&(df_hq['tradingday']<'20200824')]['changepct'].max()

-0.098019299136617555

In [48]:
df_hq[(df_hq['date_dt']==1)&(df_hq['listedsector']==6)&(df_hq['tradingday']>='20200824')]['changepct'].max()

-0.19800000000000006

In [71]:
def maxdrawdown(x):
    return np.nanmax([np.nanmax((x[i]-x[i+1:])/x[i]) for i in range(len(x)-1)])

def riskjudge0(x):
    return 1 if x>=0.4 else 0

def riskjudge1(x):
    return 1 if np.nansum(x)>=3 else 0

def riskjudge2(x):
    return 1 if np.nansum(x[:-1]*x[1:])>=1 else 0


In [75]:
def riskjudge(x):
    return  riskjudge1(x) or riskjudge2(x)

In [51]:
df_hq = df_hq.sort_values(['secucode','tradingday'])

In [60]:
df_hq.head()

tradingday secucode  closeprice  listedsector  changepct  date_dt
5590     20200103   000001       17.18             1   0.018376        0
768707   20200106   000001       17.07             1  -0.006403        0
129037   20200107   000001       17.15             1   0.004687        0
366774   20200108   000001       16.66             1  -0.028571        0
7225     20200109   000001       16.79             1   0.007803        0

In [68]:
df_hq['maxdrawdown'] = df_hq.groupby(['secucode'])['changepct'].shift(-10).rolling(10).apply(lambda x:maxdrawdown(np.cumprod(x+1))).values

In [74]:
df_hq['maxdrawdown_flag'] = df_hq['maxdrawdown'].apply(lambda x:riskjudge0(x))

In [76]:
df_hq['dt_flag'] = df_hq.groupby(['secucode'])['date_dt'].shift(-10).rolling(10).apply(lambda x:riskjudge(x)).values

In [77]:
df_hq['label'] = df_hq.apply(lambda x:x['maxdrawdown_flag'] or x['dt_flag'],axis =1)

In [78]:
df_hq['label'].value_counts()

0.0    850369
1.0      4429
Name: label, dtype: int64